# Building  Text-to-SQL capability to Amazon Athena using Amazon Bedrock

- **Use of amazon.titan-embed-text-v1 for creating embedding**
- **Use of Amazon OpenSearch as a vector database**
- **Use of anthropic.claude-v2:1 as base LLM Model**



## Contents

1. [Objective](#Objective)
1. [Background](#Background-(Problem-Description-and-Approach))
1. [Overall Workflow](#Overall-Workflow)
1. [Conclusion](#Conclusion)


## Objective

This notebook shows is to provide the code snippets within an executable flow from [this AWS Blog post](https://aws.amazon.com/blogs/machine-learning/build-a-robust-text-to-sql-solution-generating-complex-queries-self-correcting-and-querying-diverse-data-sources/).

## Background (Problem Description and Approach)

- **Problem statement**: 

Text-to-SQL solutions aim to generate SQL queries from natural language to enable non-technical users to access and analyze data. However, existing solutions face challenges related to ambiguity in natural language, needing to recreate capabilities for different databases, and collecting comprehensive metadata. The proposed solution in the text aims to address these challenges by incorporating metadata from AWS Glue Data Catalog, evaluating and correcting generated SQL queries using Amazon Athena feedback with multi-pass prompting, and leveraging Athena's support for diverse data sources.

- **Our approach**: 

[`RAG`] The [RAG approach](https://aws.amazon.com/what-is/retrieval-augmented-generation/) offers several advantages. First, it gives up-to-date, precise responses. Rather than relying only on fixed, outdated training data, RAG utilizes current external sources to formulate its answers. In this solution, we used RAG to increase the accuracy of table name from [AWS Glue Data Catalog](https://docs.aws.amazon.com/glue/latest/dg/catalog-and-crawler.html). 

[`Vector Store`] [Amazon OpenSearch](https://aws.amazon.com/opensearch-service/) offers three vector engines to choose from, each catering to different use cases.Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. This code bases used [FAISS for similiarity search](https://aws.amazon.com/about-aws/whats-new/2023/10/amazon-opensearch-service-vector-query-filters-faiss/).

[`Amazon Athena`] [Amazon Athena](https://aws.amazon.com/athena/) is a serverless, interactive analytics service built on open-source frameworks, supporting open-table and file formats. Athena provides a simplified, flexible way to analyze petabytes of data where it lives. In this solution, we used Amazon Athena as the SQL engine to 

[`Amazon Bedrock`] [Amazon Bedrock](https://aws.amazon.com/bedrock/) is a fully managed service that offers a choice of high-performing foundation models (FMs) from leading AI companies like AI21 Labs, Anthropic, Cohere, Meta, Stability AI, and Amazon via a single API, along with a broad set of capabilities you need to build generative AI applications with security, privacy, and responsible AI.

We used Bedrock with multi-step / multi-pass component which allows the LLM to correct the generated SQL query for accuracy. Here, the generated SQL is sent for syntax errors. We use Athena error messages to enrich our prompt for the LLM for more accurate and effective corrections in the generated SQL.

- *[RAG on AWS](https://aws.amazon.com/what-is/retrieval-augmented-generation/)*
- *[The langchain OpenSearch documentation](https://python.langchain.com/en/latest/ecosystem/opensearch.html)*
- *[Amazon OpenSearch service documentation](https://docs.aws.amazon.com/opensearch-service/index.html)*
- *[Amazon OpenSearch supports efficient vector](https://aws.amazon.com/about-aws/whats-new/2023/10/amazon-opensearch-service-vector-query-filters-faiss/)*



---

## Overall Workflow

**Prerequisite**

The following are prerequisites that needs to be accomplised before executing this notebook.
- A Sagemaker instance with a role having access to bedrock, glue,athena, s3,lakeformation
- Glue Database and tables. Provided spark notebook to create.
- An Amazon OpenSearch cluster for storing embeddings.Here Opensearch credenitals are in notebooks. However Opensearch cluster's access credentials (username and password) can be stored in AWS Secrets Mananger by following steps described [here](https://docs.aws.amazon.com/secretsmanager/latest/userguide/managing-secrets.html).

**The overall workflow for this notebook is as follows:**
1. Download data from source https://developer.imdb.com/non-commercial-datasets/#titleakastsvgz and upload to S3.
1. Create database and load datasets in Glue. Make sure see of the you are able to query through athena. 
1. Install the required Python packages (specifically boto version mentioned)
1. Create embedding and vector store.Do a similarity search with embeddings stored in the OpenSearch index for an input query.
1. Execute this notebook to generate sql..

## Step 1: Setup
Install the required packages.

In [2]:
# !pip3 install boto3==1.34.8
# !pip3 install jq

## Step 2: Import all modules. There are some modules in other folder.

In [3]:
import boto3
from botocore.config import Config
from langchain.llms.bedrock import Bedrock
from langchain.embeddings import BedrockEmbeddings

In [10]:
import logging 
import json
import os,sys
import re
sys.path.append("/home/ec2-user/SageMaker/llm_bedrock_v0/")
import time
import pandas as pd
import io

In [ ]:
from boto_client import Clientmodules
from llm_basemodel import LanguageModel
from athena_execution import AthenaQueryExecute
from openSearchVCEmbedding import EmbeddingBedrockOpenSearch

In [6]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

## Step 3: Checking access to Bedrock

In [7]:
session = boto3.session.Session()
bedrock_client = session.client('bedrock')
print(bedrock_client.list_foundation_models()['modelSummaries'][0])

{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large', 'modelId': 'amazon.titan-tg1-large', 'modelName': 'Titan Text Large', 'providerName': 'Amazon', 'inputModalities': ['TEXT'], 'outputModalities': ['TEXT'], 'responseStreamingSupported': True, 'customizationsSupported': [], 'inferenceTypesSupported': ['ON_DEMAND'], 'modelLifecycle': {'status': 'ACTIVE'}}


## Step 4: Invoking Athena and Bedrock

In [11]:
rqstath=AthenaQueryExecute()

athena client created 
s3 client created !!


In [ ]:
ebropen=EmbeddingBedrockOpenSearch()

## Step 5: Core logic
1. getEmbeddding : Take the input user query and vector search to find the schema from vector db created.
2. generate_sql: Taking the input prompt, generate sql . syntax_checker helps to check the sql syntax.


In [14]:
class RequestQueryBedrock:
    def __init__(self):
        # self.model_id = "anthropic.claude-v2"
        self.bedrock_client = Clientmodules.createBedrockRuntimeClient()
        self.language_model = LanguageModel(self.bedrock_client)
        self.llm = self.language_model.llm
    def getOpenSearchEmbedding(self,index_name,user_query):
        vcindxdoc=ebropen.getDocumentfromIndex(index_name=index_name)
        documnet=ebropen.getSimilaritySearch(user_query,vcindxdoc)
        return ebropen.format_metadata(documnet)
        
    def generate_sql(self,prompt, max_attempt=4) ->str:
            """
            Generate and Validate SQL query.

            Args:
            - prompt (str): Prompt is user input and metadata from Rag to generating SQL.
            - max_attempt (int): Maximum number of attempts correct the syntax SQL.

            Returns:
            - string: Sql query is returned .
            """
            attempt = 0
            error_messages = []
            prompts = [prompt]

            while attempt < max_attempt:
                logger.info(f'Sql Generation attempt Count: {attempt+1}')
                try:
                    logger.info(f'we are in Try block to generate the sql and count is :{attempt+1}')
                    generated_sql = self.llm.predict(prompt)
                    query_str = generated_sql.split("```")[1]
                    query_str = " ".join(query_str.split("\n")).strip()
                    sql_query = query_str[3:] if query_str.startswith("sql") else query_str
                    # return sql_query
                    syntaxcheckmsg=rqstath.syntax_checker(sql_query)
                    if syntaxcheckmsg=='Passed':
                        logger.info(f'syntax checked for query passed in attempt number :{attempt+1}')
                        return sql_query
                    else:
                        prompt = f"""{prompt}
                        This is syntax error: {syntaxcheckmsg}. 
                        To correct this, please generate an alternative SQL query which will correct the syntax error.
                        The updated query should take care of all the syntax issues encountered.
                        Follow the instructions mentioned above to remediate the error. 
                        Update the below SQL query to resolve the issue:
                        {sqlgenerated}
                        Make sure the updated SQL query aligns with the requirements provided in the initial question."""
                        prompts.append(prompt)
                        attempt += 1
                except Exception as e:
                    logger.error('FAILED')
                    msg = str(e)
                    error_messages.append(msg)
                    attempt += 1
            return sql_query

In [15]:
rqst=RequestQueryBedrock()

bedrock runtime client created 


In [16]:
index_name = 'llm_vector_db_metadata_indx2'

In [17]:
def userinput(user_query):
    logger.info(f'Searching metadata from vector store')
    # vector_search_match=rqst.getEmbeddding(user_query)
    vector_search_match=rqst.getOpenSearchEmbedding(index_name,user_query)
    # print(vector_search_match)
    details="It is important that the SQL query complies with Athena syntax. During join if column name are same please use alias ex llm.customer_id in select statement. It is also important to respect the type of columns: if a column is string, the value should be enclosed in quotes. If you are writing CTEs then include all the required columns. While concatenating a non string column, make sure cast the column to string. For date columns comparing to string , please cast the string input."
    final_question = "\n\nHuman:"+details + vector_search_match + user_query+ "n\nAssistant:"
    answer = rqst.generate_sql(final_question)
    return answer

## Step 6: User input in Natural Language

In [18]:
user_query='show me all the titles in US region'

In [ ]:
querygenerated=userinput(user_query)


## Step 7: Sql Query and Query Execution output

In [ ]:
import pprint
my_printer = pprint.PrettyPrinter()
my_printer.pprint(querygenerated)

In [ ]:
QueryOutput=rqstath.execute_query(querygenerated)

In [ ]:
print(QueryOutput)


## Cleanup

To avoid incurring future charges, delete the resources.


## Conclusion
In this notebook we were able to see how to use bedrock to deploy LLM Model to generate embeddings,then ingest those embeddings into OpenSearch and finally do a similarity search for user input to the documents (embeddings) stored in OpenSearch. Please read our [AWS Blog Post](https://aws.amazon.com/blogs/machine-learning/build-a-robust-text-to-sql-solution-generating-complex-queries-self-correcting-and-querying-diverse-data-sources/) on this topic to learn more about the solution.
